In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import layers
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Concatenate, Flatten, Conv2D, AveragePooling2D, MaxPool2D, Reshape, Conv2DTranspose
import os
import random
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
from tqdm import tqdm 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set seed for reproducibility
seed = 42
random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
!git clone https://github.com/femartip/Nuclei-Segmentation.git

Cloning into 'Nuclei-Segmentation'...
remote: Enumerating objects: 42082, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 42082 (delta 0), reused 2 (delta 0), pack-reused 42080
Receiving objects: 100% (42082/42082), 372.82 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (11025/11025), done.
Updating files: 100% (33893/33893), done.


In [ ]:
%cd Nuclei-Segmentation/

/content/Nuclei-Segmentation


# Load Images

In [ ]:
seed = 2023
np.random.seed = seed

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = './data/stage1_train/'
#TEST_PATH = './data/stage1_test/'

train_ids = next(os.walk(TRAIN_PATH))[1]
#test_ids = next(os.walk(TEST_PATH))[1]

X = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)



In [ ]:
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):   
    path = TRAIN_PATH + id_

    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]  
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X[n] = img

    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',  
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)  
            
    y[n] = mask 

100%|██████████| 670/670 [05:34<00:00,  2.00it/s]


# Dividing Images

In [ ]:
def create_dataset(image, kmeans, n_clusters):
    X = np.zeros((len(image), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    y = np.zeros((len(image)), dtype=np.uint8)
    for i in range(0, len(image)):
        X[i] = image[i]
        y[i] = kmeans.labels_[i]
    return X, y

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from sklearn.cluster import KMeans

model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

featureslist = []
for i in range(0, len(X)):
    try:
        img_data = np.expand_dims(X[i], axis=0)
        #print(img_data.shape)
        img_data = preprocess_input(img_data)
        features = np.array(model.predict(img_data))
        featureslist.append(features.flatten())
    except:
        continue
n_clusters = 6
# Three labels: Big, small and one nuclei
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(np.array(featureslist))

1/1 [==============================] - 0s 180ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
X_split, y_split = create_dataset(X, kmeans, n_clusters)

# Preprocessing

In [ ]:
def pipeline(X):
    X_greyscale_bright = np.zeros((len(X), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
    for i in range(0, len(X)):
        X_greyscale_bright[i] = np.dot(X[i][...,:3], [0.2989, 0.5870, 0.1140])
        X_greyscale_bright[i] = X_greyscale_bright[i] + 25

    return X_greyscale_bright

In [ ]:
X_processed = pipeline(X)
#Add one channel to the images
X_processed = np.expand_dims(X_processed, axis=3)

# Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.1, random_state=42, stratify=y_split)

# Model

In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f+y_pred_f) + smooth)

In [ ]:
def dice_coef_loss(y_true, y_pred):
    return 1.0-dice_coef(y_true, y_pred)

In [ ]:
def iou(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    iou = K.mean((intersection + 1) / (union + 1), axis=0)
    return iou

In [ ]:
def mean_iou(y_true, y_pred):
    results = []   
    for t in np.arange(0.5, 1, 0.05):
        t_y_pred = tf.cast((y_pred > t), tf.float32)
        pred = iou(y_true, t_y_pred)
        results.append(pred)
        
    return K.mean(K.stack(results), axis=0)

In [ ]:
def plot(history):
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('Loss')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['mean_iou'], label='train')
    plt.plot(history.history['val_mean_iou'], label='validation')
    plt.title('Mean IoU')
    plt.legend()
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
data  data_exploration.ipynb  model.ipynb  README.md


In [ ]:
model = tf.keras.saving.load_model("./drive/MyDrive/Colab Notebooks/model.tf", compile=False)

OSError: ignored

In [ ]:
batch_size = 64
batches_per_epoch = int(len(train_ids)/batch_size) 
epochs = 50
LR = 0.0001

model.summary()
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=dice_coef_loss, metrics=[mean_iou])

#history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=tf.keras.callbacks.TensorBoard(log_dir='./logs'))

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='model_.png',show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image(retina=True, filename='model_.png')

In [ ]:
print("Final accuracy on train set:", history.history['mean_iou'][-1])
print("Final loss on train set:", history.history['loss'][-1])
print("Final accuracy on validations set:", history.history['val_mean_iou'][-1])
print("Final loss on validations set:", history.history['val_loss'][-1])

In [ ]:
plot(history)

In [ ]:
idx = random.randint(0, len(X_train))

preds_train = model.predict(X_train[idx:idx+1], verbose=1)

#Postprocessing threshold > 0.5
preds_train_t = (preds_train > 0.5).astype(np.uint8)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.title('Original image')
plt.imshow(X_train[idx])
plt.subplot(1,3,2)
plt.title('Ground truth mask')
plt.imshow(np.squeeze(y_train[idx]))
plt.subplot(1,3,3)
plt.title('Predicted mask')
plt.imshow(np.squeeze(preds_train_t))
plt.show()